In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

In [31]:
NUM_SAMPLES = 10000    
MAX_SEQ_LEN = 7        
VOCAB_SIZE = 11         
EMBED_DIM = 32
LATENT_DIM = 64
BATCH_SIZE = 64
EPOCHS = 20

In [32]:
def generate_data(num_samples=10000, max_len=7, vocab_size=11):
    X, Y = [], []
    for _ in range(num_samples):
        seq_len = np.random.randint(2, max_len + 1)
        seq = np.random.randint(1, vocab_size - 1, size=seq_len).tolist()
        X.append(seq)
        Y.append(seq[::-1])
    return X, Y

input_texts, target_texts = generate_data(NUM_SAMPLES, MAX_SEQ_LEN, VOCAB_SIZE)

In [33]:
START_TOKEN = VOCAB_SIZE - 2
END_TOKEN = VOCAB_SIZE - 1

encoder_input = tf.keras.preprocessing.sequence.pad_sequences(input_texts, maxlen=MAX_SEQ_LEN, padding='post')

decoder_input = [[START_TOKEN] + seq for seq in target_texts]
decoder_output = [seq + [END_TOKEN] for seq in target_texts]

decoder_input = tf.keras.preprocessing.sequence.pad_sequences(decoder_input, maxlen=MAX_SEQ_LEN+1, padding='post')
decoder_output = tf.keras.preprocessing.sequence.pad_sequences(decoder_output, maxlen=MAX_SEQ_LEN+1, padding='post')

In [34]:
encoder_inputs = layers.Input(shape=(MAX_SEQ_LEN,))
enc_emb = layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(encoder_inputs)
encoder_lstm = layers.LSTM(LATENT_DIM, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = layers.Input(shape=(MAX_SEQ_LEN+1,))
dec_emb = layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(decoder_inputs)
decoder_lstm = layers.LSTM(LATENT_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = layers.Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_17      │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, 7, 32)     │        352 │ input_layer_16[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_10        │ (None, 7)         │          0 │ input_layer_16[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, 8, 32)     │        352 │ input_layer_17[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ [(None, 64),      │     24,832 │ embedding_10[0][… │
│                     │ (None, 64),       │            │ not_equal_10[0][… │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ [(None, 8, 64),   │     24,832 │ embedding_11[0][… │
│                     │ (None, 64),       │            │ lstm_6[0][1],     │
│                     │ (None, 64)]       │            │ lstm_6[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 8, 11)     │        715 │ lstm_7[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 51,083 (199.54 KB)

 Trainable params: 51,083 (199.54 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

history = model.fit(
    [encoder_input, decoder_input],
    np.expand_dims(decoder_output, -1),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=0
)

In [36]:
encoder_model = models.Model(encoder_inputs, encoder_states)

decoder_state_input_h = layers.Input(shape=(LATENT_DIM,))
decoder_state_input_c = layers.Input(shape=(LATENT_DIM,))
dec_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding_layer = layers.Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)
dec_emb2 = decoder_embedding_layer(decoder_inputs)

decoder_lstm_layer = decoder_lstm
decoder_dense_layer = decoder_dense

dec_outputs2, state_h2, state_c2 = decoder_lstm_layer(dec_emb2, initial_state=dec_states_inputs)
dec_states2 = [state_h2, state_c2]
dec_outputs2 = decoder_dense_layer(dec_outputs2)

decoder_model = models.Model(
    [decoder_inputs] + dec_states_inputs,
    [dec_outputs2] + dec_states2
)

In [37]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = START_TOKEN
    stop_condition = False
    decoded = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == END_TOKEN or len(decoded) >= MAX_SEQ_LEN:
            stop_condition = True
        else:
            decoded.append(sampled_token_index)

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded


In [38]:
test_seq = np.array([ [1, 5, 7, 3, 0,0,0] ])
decoded = decode_sequence(test_seq)
print("Input :", test_seq[0])
print("Predicted reversed:", decoded)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Input : [1 5 7 3 0 0 0]
Predicted reversed: [np.int64(3), np.int64(7), np.int64(7), np.int64(7), np.int64(7), np.int64(7), np.int64(7)]
